In [7]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from app import app, db, Company, Acquisition
import pandas as pd
import datetime

In [8]:
# IMPORTANT NOTE:
# Data is from https://www.kaggle.com/datasets/joebeachcapital/technology-mergers-and-acquisitions
# The data provided on kaggle does NOT contain the required lattitude and longitude data.
# To add the lattitude and longitude data, I used https://www.geoapify.com/tools/geocoding-online
# Note that with a free sub, this requires you to manually do it in batches of 500
# Ideally this would be replaced with a proper API for geocoding, but this is a quick and dirty solution

acquisitions = pd.read_csv('data/acquisitions.csv', header=0, delimiter=',')
acquired = pd.read_csv('data/acquired.csv', header=0, delimiter=',')
acquiring = pd.read_csv('data/acquiring.csv', header=0, delimiter=',')

In [9]:
print("Acquired headers: ", acquired.columns)
print("Acquiring headers: ", acquiring.columns)

Acquired headers:  Index(['Company', 'CrunchBase Profile', 'Image', 'Tagline', 'Year Founded',
       'Market Categories', 'Address (HQ)', 'City (HQ)', 'State / Region (HQ)',
       'Country (HQ)', 'lat', 'lon', 'Description', 'Homepage', 'Twitter',
       'Acquired by', 'Acquisitions ID', 'API'],
      dtype='object')
Acquiring headers:  Index(['Acquiring Company', 'CrunchBase Profile', 'Image', 'Tagline',
       'Market Categories', 'Year Founded', 'IPO', 'Founders',
       'Number of Employees', 'Number of Employees (year of last update)',
       'Total Funding ($)', 'Number of Acquisitions', 'Board Members',
       'Address (HQ)', 'City (HQ)', 'State / Region (HQ)', 'Country (HQ)',
       'Description', 'Homepage', 'Twitter', 'Acquired Companies',
       'Acquisitions ID', 'API', 'lat', 'lon'],
      dtype='object')


In [10]:
with app.app_context():

    for index, row in acquiring.iterrows():
        company = Company(
            name = row['Acquiring Company'],
            crunchbase_url = row['CrunchBase Profile'],
            image_url = row['Image'],
            tagline = row['Tagline'],
            year_founded = row['Year Founded'],
            year_ipo = row['IPO'],
            number_of_employees = row['Number of Employees'],
            market_category = row['Market Categories'],
            address = row['Address (HQ)'],
            description = row['Description'],
            website = row['Homepage'],
            longitude = row['lon'],
            latitude = row['lat']
        )

        if not Company.query.filter_by(name=company.name).first():
            db.session.add(company)
            db.session.commit()

In [11]:
with app.app_context():
    for index, row in acquired.iterrows():
        company = Company(
            name = row['Company'],
            crunchbase_url = row['CrunchBase Profile'],
            image_url = row['Image'],
            tagline = row['Tagline'],
            year_founded = row['Year Founded'],
            year_ipo = None,
            number_of_employees = None,
            market_category = row['Market Categories'],
            address = row['Address (HQ)'],
            description = row['Description'],
            website = row['Homepage'],
            longitude = row['lon'],
            latitude = row['lat']
        )
        
        if not Company.query.filter_by(name=company.name).first():
            db.session.add(company)
            db.session.commit()

In [12]:
# get first 10 rows of companies
with app.app_context():
    companies = Company.query.all()
    print("Total number of companies: ", len(companies))
    print("\nFirst 10 companies: ")
    for company in companies[:10]:
        print(company.name)

Total number of companies:  1646

First 10 companies: 
Adobe
Amazon
AOL
Apple
AT&T
BlackBerry
CA Technologies
Cisco Systems
Comcast
Dell


In [13]:
print("Acquisition headers: ", acquisitions.columns)

Acquisition headers:  Index(['Title', 'Acquired Company', 'Acquiring Company',
       'Year of acquisition announcement', 'Deal announced on', 'Price',
       'Status', 'Terms', 'Acquisition Profile', 'News', 'News Link'],
      dtype='object')


In [14]:
for index, row in acquisitions.iterrows():
    with app.app_context():
        purchased_company = Company.query.filter_by(name=row['Acquired Company']).first()
        purchasing_company = Company.query.filter_by(name=row['Acquiring Company']).first()
        
        if not purchased_company or not purchasing_company:
            print("Error adding acquisition: ")
            print("Purchased company: ", row['Acquired Company'])
            print("Purchasing company: ", row['Acquiring Company'])
            print()
            continue
        
        date = row['Deal announced on'],
        if date:
            date = datetime.datetime.strptime(date[0], '%d/%m/%Y')
            
        
        acquisition = Acquisition(
            purchased_company = purchased_company,
            purchasing_company = purchasing_company,
            purchased_company_id = purchased_company.id,
            purchasing_company_id = purchasing_company.id,
            deal_date = date,
            deal_value = row['Price'],
            deal_status = row['Status'],
            deal_terms = row['Terms'],
            news_url = row['News Link'],
            news_title = row['News']
        )
        
        if not Acquisition.query.filter_by(purchased_company_id=acquisition.purchased_company_id, purchasing_company_id=acquisition.purchasing_company_id).first():
            db.session.add(acquisition)
            db.session.commit()
            

Error adding acquisition: 
Purchased company:  Tellme
Purchasing company:  [24]7



/var/folders/p4/bq3lmyls11s56yy7r2vv6c0r0000gn/T/ipykernel_36785/205144347.py:31: SAWarning: Object of type <Acquisition> not in session, add operation along 'Company.acquisitions_as_purchased' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  if not Acquisition.query.filter_by(purchased_company_id=acquisition.purchased_company_id, purchasing_company_id=acquisition.purchasing_company_id).first():
/var/folders/p4/bq3lmyls11s56yy7r2vv6c0r0000gn/T/ipykernel_36785/205144347.py:31: SAWarning: Object of type <Acquisition> not in session, add operation along 'Company.acquisitions_as_purchasing' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  if not Acquisition.query.filter_by(purchased_company_id=acquisition.purchased_company_id, purchasing_company_id=acquisition.purchasing_comp

Error adding acquisition: 
Purchased company:  Picasa
Purchasing company:  Appirio

Error adding acquisition: 
Purchased company:  delicious
Purchasing company:  AVOS Systems

Error adding acquisition: 
Purchased company:  Trapeze Networks
Purchasing company:  Belden

Error adding acquisition: 
Purchased company:  ICQ
Purchasing company:  Digital Sky Technologies

Error adding acquisition: 
Purchased company:  GoPago
Purchasing company:  Doubleeam

Error adding acquisition: 
Purchased company:  EDS
Purchasing company:  General Motors

Error adding acquisition: 
Purchased company:  Distill
Purchasing company:  Gild

Error adding acquisition: 
Purchased company:  VeriFone
Purchasing company:  Gores Technology Group

Error adding acquisition: 
Purchased company:  DoubleClick
Purchasing company:  Hellman & Friedman

Error adding acquisition: 
Purchased company:  Medio
Purchasing company:  HERE

Error adding acquisition: 
Purchased company:  eVoice
Purchasing company:  J2 Global Communicati

In [15]:
with app.app_context():
    acquisitions = Acquisition.query.all()
    print("Total number of acquisitions: ", len(acquisitions))
    print("\nFirst 10 acquisitions: ")
    for acquisition in acquisitions[:10]:
        print(acquisition.purchased_company.name, " acquired by ", acquisition.purchasing_company.name)

Total number of acquisitions:  1620

First 10 acquisitions: 
Palm  acquired by  3Com
Accelio Corporation  acquired by  Adobe
Aldus Corp  acquired by  Adobe
Amicima  acquired by  Adobe
Ares Software  acquired by  Adobe
Auditude  acquired by  Adobe
Behance  acquired by  Adobe
BluePoint Technologies  acquired by  Adobe
Day Software  acquired by  Adobe
Demdex  acquired by  Adobe
